# Download from Kaggle

This dataset contains 1.3M Linkedin job posting data scraped in 2024.
It consists of three csv tables:
* job_skills.csv
* job_summary.csv
* linkedin_job_postings.csv

Dataset: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024/data

In [1]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("asaniczka/1-3m-linkedin-jobs-and-skills-2024")

print("Path to dataset files:", path)

print("Files:")
for filename in os.listdir(path):
    print(filename)

Path to dataset files: /kaggle/input/1-3m-linkedin-jobs-and-skills-2024
Files:
job_summary.csv
job_skills.csv
linkedin_job_postings.csv


# Start Spark Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("CS774 data wrangling") \
    .config("spark.driver.memory", "9g")\
    .config("spark.driver.maxResultSize", "6g")\
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [3]:
# df_test = spark.read.csv('/content/drive/MyDrive/part-00000-2b356652-1f83-4aac-b785-0b3a784f75f9-c000.csv', header=True)
# df = df_test.toPandas()
# print(df.shape)
# df.head(10)

# Read the csv datasets as dataframes

In [4]:
from pyspark.sql.functions import regexp_replace

df_job_skills = spark.read.csv(path+'/job_skills.csv', header=True)
df_job_summary = spark.read.option("multiLine", True)\
                           .option("quote", '"')\
                           .option("escape", '"')\
                           .option("header", True)\
                           .option("encoding", "UTF-8")\
                           .csv(path + '/job_summary.csv')\
                           .withColumn("job_summary", regexp_replace("job_summary", r"\n", " "))\
                           .withColumn("job_summary", regexp_replace("job_summary", r"\"", " "))
df_linkedin_job_postings = spark.read.csv(path+'/linkedin_job_postings.csv', header=True)

df_job_skills.show(n=5, truncate=False)
df_job_summary.show(n=5, truncate=False)
df_linkedin_job_postings.show(n=5, truncate=False)

print(f"job_skills.csv row count: {df_job_skills.count()}")
print(f"job_summary.csv row count: {df_job_summary.count()}")
print(f"linkedin_job_postings.csv row count: {df_linkedin_job_postings.count()}")

+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|job_link                                                                                                                       |job_skills                                              

# Filter out tech jobs

Prompt ChatGPT: I'm trying to find all job titles relevant to tech industry according to Linkedin scraping data here: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024/data
Can you help identify the keywords I can use to filter job title?

In [5]:
from pyspark.sql.functions import lower, col

tech_keywords = [
    'software', 'developer', 'engineer', 'programmer', 'coder', 'full stack',
    'front end', 'back end', 'devops', 'cloud', 'aws', 'azure', 'gcp',
    'infrastructure', 'site reliability', 'sre', 'system administrator', 'sysadmin',
    'data', 'scientist', 'analyst', 'ml', 'machine learning', 'ai', 'artificial intelligence',
    'nlp', 'deep learning', 'computer vision', 'analytics', 'statistician',
    'security', 'infosec', 'cybersecurity', 'soc analyst', 'penetration tester',
    'ethical hacker', 'privacy', 'web developer', 'web engineer', 'frontend',
    'backend', 'mobile developer', 'android', 'ios', 'flutter', 'react native',
    'product manager', 'technical program manager', 'scrum master', 'agile coach',
    'project manager', 'qa', 'test engineer', 'tester', 'quality assurance',
    'platform engineer', 'ci/cd', 'ux', 'ui', 'user experience', 'product designer',
    'interaction designer', 'ux researcher', 'it support', 'technical support',
    'help desk', 'desktop support', 'it technician'
]


pattern = '|'.join([kw.replace(' ', r'\s') for kw in tech_keywords])

df_tech_job = df_linkedin_job_postings.withColumn("job_title", lower(col("job_title"))) \
                    .filter(col("job_title").rlike(f"(?i){pattern}"))

# df_tech_job.select("job_title").show(20, truncate=False)
df_tech_job.count()

303968

# Data preprocessing: examine why the number of unique job_summary is less than the number of unique job link
Intuitively, each job posting should have different job description. We observe that the unique job description count is much less than the total job postings.

In [6]:
from pyspark.sql.functions import col

# find top 10 job_summary values
top_job_summaries = df_job_summary.groupBy("job_summary") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(10) \
    .select("job_summary")

# join with original dataframe
top_rows = df_job_summary.join(top_job_summaries, on="job_summary", how="inner")

top_rows.show(10, truncate=False)
# top_rows.coalesce(1).write.option("header", True).csv("./output/top_rows", mode='overwrite')

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

We can see that companies such as Walmart post the same job while at different locations multiple time using exactly the same job description. Given such, we can simply drop the duplicates based on job_summary.

In [7]:
# drop the duplicate based on job_summary
df_job_summary_cleaned = df_job_summary.dropDuplicates(['job_summary'])
# df_job_summary_cleaned.cache()
# n_before = df_job_summary.count()
# n_after = df_job_summary_cleaned.count()

# print(f"Before dropping duplicate: {n_before}")
# print(f"After dropping duplicate: {n_after}")

# Get frequency stats for all job skills

In [8]:
from pyspark.sql.functions import split, explode, trim, col, lower
df_tech_job_skills = df_job_skills.join(df_tech_job, 'job_link', how="inner") \
                                  .select("job_skills", "job_link")
df_trimmed = (
            df_tech_job_skills
            .withColumn("job_skills", split(col("job_skills"), ", "))  # skill column will contain a list of skills
            .withColumn("job_skills", explode(col("job_skills"))) # explode list of one row into multiple rows
            .withColumn("job_skills", lower(col("job_skills"))) # convert to lowercase (82699 -> 73891)
            # .withColumn("job_skills", regexp_replace(col("job_skills"), r"\s+skills?$", "")) # remove 'skill', 'skills'
            .withColumn("job_skills", trim(col("job_skills")))  # trim the leading and trailing spaces
          )
df_trimmed.show(10, truncate=False)

df_count = df_trimmed.groupBy("job_skills").count().orderBy("count", ascending=False)
df_count.cache()
df_count.show(10, truncate=False)

df_count.coalesce(1).write.option("header", True).csv("./output/group_by_stats", mode='overwrite')
print("There are ", df_count.count(), " different skills in total.")

+----------------------+----------------------------------------------------------------------------------+
|job_skills            |job_link                                                                          |
+----------------------+----------------------------------------------------------------------------------+
|project management    |https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|mechanical engineering|https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|hvac                  |https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|mep                   |https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|site supervision      |https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|project commissioning |https://ae.linkedin.com/jobs/view/project-operations-engineer-at-taqeef-3803932295|
|inspection            |http

# Label pruning: only keep top K skills based on frequency
We observe that there are much more skills than job listing (73891 v.s. 10000!). Given such sparse data (lable), we decide to prune low frequency labels so that our model can better fit.

# Synonym transformation
We are also doing synonym transformation to group similar skills together.

In [12]:
K = 500

top_skills = df_count.limit(int(K))\
                      .select("job_skills").rdd.flatMap(lambda x: x).collect()
df_filtered = df_trimmed.filter(col("job_skills").isin(top_skills))


import pandas as pd
from pyspark.sql.functions import create_map, lit, coalesce, col
from itertools import chain

synonym_df = pd.read_csv('/content/Tskills_synonyms_new.csv')

skill_map = {}
for _, row in synonym_df.iterrows():
    target = row['target_skills'].strip()
    if pd.notnull(row['Synonyms']):
        synonyms = [s.strip() for s in row['Synonyms'].split(',')]
        for syn in synonyms:
            skill_map[syn] = target

mapping_expr = create_map([lit(x) for x in chain(*skill_map.items())])

df_filtered_mapped = df_filtered.withColumn(
    "job_skills", coalesce(mapping_expr.getItem(col("job_skills")), col("job_skills"))
)
df_filtered_count = df_filtered_mapped.groupBy("job_skills").count().orderBy("count", ascending=False)
df_filtered_count.coalesce(1).write.option("header", True).csv("./output/skills_kept_synonym", mode='overwrite')

from pyspark.sql.functions import collect_list, concat_ws
df_skills_preprocessed = df_filtered_mapped.groupBy("job_link").agg(
    concat_ws(", ", collect_list("job_skills")).alias("job_skills")
)
df_skills_preprocessed.show(10, truncate=False)
# df_skills_preprocessed.coalesce(1).write.option("header", True).csv("./output/cleaned", mode='overwrite')

/usr/local/lib/python3.11/dist-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+--------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|job_link                                                                                                                              |job_skills                                                                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# pdf_count = df_count.toPandas()
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.bar(pdf_count["job_skills"], pdf_count["count"])
# plt.xticks(rotation=45, ha='right')
# plt.xlabel("Skill")
# plt.ylabel("Count")
# plt.title("Skills Distribution by Frequency")
# plt.tight_layout()
# plt.show()

# Join skills with summary to create training data for our model
Finally, we can generate our training data. By joining skills with job_summary using left join, we can exclude jobs that contains no skills after pruning.

**Schema**

| filtered_skills | job_summary |

In [13]:
# subset = 10000

df_result = df_skills_preprocessed.join(df_job_summary_cleaned, 'job_link')
df_result.cache()
df_result.show(10, truncate=False)
filtered_df = df_result.select("job_link", "job_skills", "job_summary")
# filtered_df.coalesce(1).write.option("header", True).csv("./output/subset", mode='overwrite')

+------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Join skills with tech job to filter only tech job postings

In [14]:
df_tech_result = filtered_df.join(df_tech_job, 'job_link', how="inner")
df_tech_result.cache()
print(df_tech_result.count())
filtered_tech_df = df_tech_result.select("job_skills", "job_summary")
filtered_tech_df.cache()
print(filtered_tech_df.count())
filtered_tech_df.coalesce(1).write.option("header", True).csv("./output/subset", mode='overwrite')

213139
213139


In [15]:
filtered_tech_df.show()

+--------------------+--------------------+
|          job_skills|         job_summary|
+--------------------+--------------------+
|problem solving, ...|Location: Melbour...|
|project managemen...|Our Water team ta...|
|data analysis, st...|About Syrah Syrah...|
|power bi, tableau...|Be a business par...|
|customer service,...|Absolutely love s...|
|data management, ...|Job Description J...|
|teamwork, interpe...|What do we do? Ou...|
|manufacturing, pr...|Sterling Pumps ar...|
|engineering, proj...|The Role If leadi...|
|engineering, engi...|Advancing careers...|
|            teamwork|Employment Type :...|
|tableau, inventor...|Description A bri...|
|maintenance, proc...|THIS OPENS THE DO...|
|maintenance, tech...|About The Role Re...|
|devops, devops, s...|This is a Permane...|
|quality assurance...|Position Overview...|
|interviewing, int...|Our Recruitment T...|
|data analysis, ri...|Primary Details T...|
|engineering, proj...|Senior Civil Engi...|
|security, cissp, ...|Role withi

In [ ]:
# # To inspect the data, we use pandas to read the first 100 rows

# import pandas as pd

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# # You might need to adjust the filename if it's different.
# file_path = "./output/subset/part-00000-2a3b728d-3236-4dbb-82cc-8c2392a89cc0-c000.csv"

# try:
#     df = pd.read_csv(file_path)
#     print(df.head(100))
#     print(df.shape)
# except FileNotFoundError:
#     print(f"Error: File not found at {file_path}. Check the file path and name.")
# except pd.errors.EmptyDataError:
#     print(f"Error: The CSV file at {file_path} is empty.")
# except pd.errors.ParserError:
#     print(f"Error: Could not parse the CSV file at {file_path}. Check the file format.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")
